In [1]:
import h5pyd
import numpy as np

In [2]:
DOMAIN_PATH = "/home/test_user1/test/refresh_test.h5"
f = h5pyd.File(DOMAIN_PATH, "w", libver="latest")

In [3]:
# create an extendable dataset
dset_f = f.create_dataset("dset", (3,4), maxshape = (None, None), dtype=np.int32)
dset_f

<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [4]:
f.swmr_mode = True   # not strictly needed for HSDS, but compatible with h5py usage

In [5]:
g = h5pyd.File(DOMAIN_PATH, swmr_mode=True)  # read-only file handles need to set swmr mode in the constructor

In [6]:
dset_g = g["dset"]  # get a reference to the dataset via the read-only file handle
dset_g

<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [7]:
dset_f.resize((6, 8))  # Resize the dataset using the writable file handle

In [8]:
dset_g  # the read-only reference to the dataset doesn't "see" the shape change yet

<HDF5 dataset "dset": shape (3, 4), type "<i4">

In [9]:
dset_g.refresh()  # this will re-synch this reference to the dataset to the server's view
dset_g

<HDF5 dataset "dset": shape (6, 8), type "<i4">

In [10]:
dset_f.attrs["a1"] = "an attribute"  # create an attribute

In [11]:
"a1" in dset_g.attrs  # don't need a refresh to see the attribute change from the read-only handle

True

In [12]:
# create a sub-group
f.create_group("subgrp")

<HDF5 group "/subgrp" (0 members)>

In [13]:
"subgrp" in g  # doesn't show up in the g reference

False

In [14]:
g.refresh()  # after a refresh, subgroup will be visibile
"subrp" in g

False